<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/16_vit/01_vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Transformer

---

## 目的

CIFAR-10を用いて，フルスクラッチで学習したViTとCNNの比較を行います．また，ImageNetで事前学習をしたモデルを用いてファインチューニングを行います．

## 必要なモジュールの読み込み

まず，Colaboratoryにないパッケージをインストールします．timmにはViTやSwinなどのネットワークだけでなく，様々な最適化手法や学習率のスケジューラーが用意されています．

In [ ]:
!pip -q install timm==0.5.4

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import timm
from timm.models.layers import trunc_normal_, DropPath
from timm.models import create_model
from timm.scheduler.cosine_lr import CosineLRScheduler
from functools import partial
from time import time

## ネットワークの定義

Vision Transformer (ViT) [1] はTransformerをコンピュータビジョンに応用した画像分類手法です．ViTは入力画像を固定領域のパッチに分割して埋め込み層を介して，Transformer Encoderに入力します．Transformer Encoder内のSelf Attentionでパッチの関係を学習することで，畳み込みニューラルネットワーク (CNN) とは異なり，浅い層から画像全体の特徴を捉えられます．これにより，ImageNetなどのクラス分類タスクでCNNの性能を上回りました．また，ViTはセマンティックセグメンテーションや動画像認識などのタスクに応用され，CNNベースの性能を上回りました．

<img src="https://github.com/ShokiSuzuki/MPRGDeepLearningLectureNotebook/blob/dev/16_vit/model_scheme.png?raw=true" width=60%>


### ファインチューニング

ViTは大規模データセットで事前学習し，小規模データセットでファインチューニングすることが効果的です．事前学習の画像枚数を変更すると，CNNは枚数を多くしても精度に限界がある一方で，ViTは枚数が多いほど精度向上が見込めます．ViTはJFT-300Mという3億枚の画像が含まれているデータセットで事前学習し，様々なデータセットでファインチューニングをすることでSoTAを達成していますが，非公開のデータセットのため再現不可能です．


### 各モジュールの実装

以下では，ViTを構成する主要なモジュールについて一つづつ実装を行なっていきます．

### Patch Embedding

Patch Embeddingは，入力画像を固定領域のパッチに分割して埋め込む処理を行います．例えば，$224 \times 224$ピクセルの画像を入力として各パッチのサイズを$16 \times 16$ピクセルとした場合，重なり合わないように$14 \times 14$の領域に分割します．分割されたパッチは，それぞれflatにして全結合に入力することで埋め込みます．また，学習可能なパラメータであるクラストークンを結合します．

基本的にはパッチをflatにして全結合に入力しますが，実装上は，カーネルサイズ（パッチサイズ）= ストライドとした2次元畳み込みでも可能です．






In [ ]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        self.img_size    = img_size
        self.patch_size  = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        # 埋め込み処理のための重み
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        return self.proj(x).flatten(2).transpose(1, 2)


### Position Embedding

Position Embeddingは，パッチの位置情報を学習するパラメータです．このパラメータは，Patch Embeddingのあとにそれぞれのパッチに足されます．ネットワークが学習する過程で位置情報を獲得するため，学習条件で値が変化します．

Patch EmbeddingとPosition Embeddingを定式化すると以下のようになります．

\begin{aligned}
\mathbf{z}_0 &= [ \mathbf{x}_\text{class}; \, \mathbf{x}^1_p \mathbf{E}; \, \mathbf{x}^2_p \mathbf{E}; \cdots; \, \mathbf{x}^{N}_p \mathbf{E} ] + \mathbf{E}_{pos},
&& \mathbf{E} \in \mathbb{R}^{(P^2 \cdot C) \times D},\, \mathbf{E}_{pos}  \in \mathbb{R}^{(N + 1) \times D}
\end{aligned}

ここで，$\mathbf{x}_\text{class}$はクラストークン，$\mathbf{x}_p$はパッチ，$N$はパッチ数，$P$はパッチサイズ，$C$はチャンネル数，$D$は埋め込み次元数，$\mathbf{E}$は全結合，$\mathbf{E}_{pos}$はPosition Embeddingです．



### Multi-Head Attention

Self-Attentionはパッチトークンを空間方向に混ぜるような変換を行います．Multi-Head Attentionはパッチトークンをベクトルのdepth方向に$h$個に分割し，それぞれでSelf-Attentionを求めます．例えばSmallモデルの場合，埋め込み次元数が384でhead数が6であるため，64次元のベクトルが6つある状態になります．
これにより，head毎に注目したパッチトークンが異なる特徴が得られるため，アンサンブル効果による精度向上が見込めます．

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        assert dim % num_heads == 0, "dim should be divisible by num_heads"
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape

        # パッチトークンをQ, K, Vに変換し，それぞれベクトルのdepth方向に分割
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)

        # Attention Weightの算出
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        # Attention WeightとVを乗算
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

### Multi-Layer Perceptron

Multi-Head Attentionでは空間方向に混ぜるような変換を行うのに対し，Multi-Layer Perceptronではベクトルのdepth方向に混ぜるような変換を行います．活性化関数にはGELUを使用します．

In [ ]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features

        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.drop1 = nn.Dropout(drop)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop2 = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.drop2(x)
        return x

### Transformer Encoder

Transformer Encoderは，Multi-Head AttentionとMulti-Layer Perceptronを交互に使用します．また，それぞれResidual Connectionを用います．

In [ ]:
class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, drop=0.,
                 attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path1 = DropPath(drop_path) if drop_path > 0. else nn.Identity()

        self.norm2 = norm_layer(dim)
        self.mlp = Mlp(in_features=dim, hidden_features=int(dim * mlp_ratio), act_layer=act_layer, drop=drop)
        self.drop_path2 = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        x = x + self.drop_path1(self.attn(self.norm1(x)))
        x = x + self.drop_path2(self.mlp(self.norm2(x)))
        return x

### ネットワーク全体の構築

これまで定義したクラスをもとにViTを構築します．

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., qkv_bias=True,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0., embed_layer=PatchEmbed,
                 norm_layer=None, act_layer=None, block_fn=Block):
        super().__init__()
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer = act_layer or nn.GELU

        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim

        # Patch Embeddingの定義
        self.patch_embed = embed_layer(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim,
        )
        num_patches = self.patch_embed.num_patches

        # クラストークンの定義
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))

        # Position Embeddingの定義 (クラストークンのためにパッチ数+1)
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop  = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth

        # Transformer Encoderの定義
        self.blocks = nn.Sequential(*[
            block_fn(
                dim=embed_dim,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[i],
                norm_layer=norm_layer,
                act_layer=act_layer
            )
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # Classifier Headの定義
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        B = x.shape[0]

        # Patch Embedding
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)

        # クラストークンの結合
        x = torch.cat((cls_tokens, x), dim=1)

        # Position Embeddingの加算
        x = x + self.pos_embed
        x = self.pos_drop(x)

        # Transformer Encoderへ入力
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        # 0番目にあるクラストークンを取り出して全結合へ入力
        x = self.head(x[:, 0])
        return x

ViTでは，いくつかのネットワークサイズが提案されています．

以下では，上で定義したVisionTransformerクラスを活用して，それらを呼び出せるような関数を定義します．

In [ ]:
def vit_tiny_patch16_224(pretrained=False, patch_size=16, num_heads=3, **kwargs):
    model = VisionTransformer(
        patch_size=patch_size, num_heads=num_heads, embed_dim=192, depth=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model

def vit_small_patch16_224(pretrained=False, patch_size=16, num_heads=6, **kwargs):
    model = VisionTransformer(
        patch_size=patch_size, num_heads=num_heads, embed_dim=384, depth=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model

def vit_base_patch16_224(pretrained=False, patch_size=16, num_heads=12, **kwargs):
    model = VisionTransformer(
        patch_size=patch_size, num_heads=num_heads, embed_dim=768, depth=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model

## フルスクラッチでの学習

まず，フルスクラッチ（乱数で初期化されたパラメータ）から学習します．


### データの準備




In [ ]:
img_size = 32

# 学習用データのデータ増幅と評価用データのデータ増幅の設定
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.Resize(img_size),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                     ])

test_transform  = transforms.Compose([transforms.Resize(img_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                     ])

# 学習用データと評価用データの用意
dataset_train = torchvision.datasets.CIFAR10("./", train=True, transform=train_transform, download=True)
dataset_test  = torchvision.datasets.CIFAR10("./", train=False, transform=test_transform, download=False)

# 学習用データのDataloaderと評価用データのDataloaderの用意
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=128, num_workers=2, pin_memory=True, drop_last=True)
dataloader_test  = torch.utils.data.DataLoader(dataset_test, batch_size=64, num_workers=2, pin_memory=True, drop_last=False)

### 学習条件の設定

ViTの性能をCNNと比較するために，ViTのSmallモデルとパラメータ数が同等のResNet-50を用います．今回はViTとCNNの学習条件を揃えて学習します．
ViTを学習させるときのパッチサイズは4とします．「データの準備」で画像サイズを32と設定したため，パッチ数は$8\times 8=64$となります．

In [ ]:
# クラス数の設定
num_classes = 10

# ViTの定義
vit = vit_small_patch16_224(pretrained=False, num_classes=num_classes, img_size=img_size, patch_size=4, num_heads=6)
# CNNの定義（パラメータ数比較用）
cnn = create_model("resnet50", pretrained=False, num_classes=num_classes)

# 学習率の設定
lr  = 0.0005
# Weight Decayの設定
weight_decay = 0.05
# エポック数の設定
epochs = 10
# Warmup Epochの設定
warmup_t = 3

# 最適化手法の設定 (ViT)
optimizer_vit     = torch.optim.AdamW(vit.parameters(), lr=lr, weight_decay=weight_decay)
# 学習率のスケジューラーの設定 (ViT)
lr_scheduler_vit  = CosineLRScheduler(optimizer=optimizer_vit, t_initial=epochs, warmup_t=warmup_t)

ViTとCNNのパラメータ数を確認します．

In [ ]:
def num_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("ViT parameters: ", num_parameters(vit))
print("CNN parameters: ", num_parameters(cnn))

### ViTの学習と評価


#### 参考精度

デフォルトの設定で学習を行なった場合に達成できる分類精度（目安）を記載します．

※ 学習が演習時間内に終了しない場合を考慮して記載しています．

**CNNの参考分類精度（スクラッチ）**

```
epoch: 10, mean loss: 0.725, mean accuracy: 0.74, elapsed_time : 213.11
test accuracy: 0.7433
```

**ViTの参考分類精度（スクラッチ）**

```
epoch: 10, mean loss: 0.87, mean accuracy: 0.69, elapsed_time : 240.45
test accuracy: 0.6954
```



In [ ]:
# 誤差関数の設定
criterion = torch.nn.CrossEntropyLoss()

# ネットワークをGPUへ
device = torch.device("cuda")
vit.to(device)

# 学習時間短縮のためにAutomatic Mixed Precision(amp)を利用
use_amp = True
scaler_vit = torch.cuda.amp.GradScaler(enabled=use_amp)

start = time()
for epoch in range(epochs):
    # ネットワークを学習モードへ変更
    vit.train()

    # ログ用の設定
    sum_loss = 0.0
    count    = 0

    for img, cls in dataloader_train:
        # 学習用データをGPUへ
        img = img.to(device, non_blocking=True)
        cls = cls.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            # ネットワークに学習用データを入力
            logit = vit(img)
            # 損失を計算
            loss  = criterion(logit, cls)

        # ネットワークの更新
        optimizer_vit.zero_grad()
        scaler_vit.scale(loss).backward()
        scaler_vit.step(optimizer_vit)
        scaler_vit.update()

        # ログ用に損失値と正解したデータ数を取得
        sum_loss += loss.item()
        count    += torch.sum(logit.argmax(dim=1) == cls).item()

    # 学習率の調整
    lr_scheduler_vit.step(epoch)

    # ログの表示
    print(f"epoch: {epoch+1}, ", end='')
    print(f"mean loss: {round(sum_loss/len(dataloader_train), 3)}, ", end='')
    print(f"mean accuracy: {round(count/len(dataloader_train.dataset), 2)}, ", end='')
    print(f"elapsed_time : {round(time()-start, 2)}")

    # ネットワークを評価モードへ変更
    vit.eval()

    # ログ用の設定
    count = 0

    # ネットワークの評価
    with torch.no_grad():
        for img, cls in dataloader_test:
            # 評価用データをGPUへ
            img = img.to(device, non_blocking=True)
            cls = cls.to(device, non_blocking=True)

            # ネットワークに評価用データを入力
            logit = vit(img)

            # 正解したデータ数をカウント
            count += torch.sum(logit.argmax(dim=1) == cls).item()

        # 評価用データに対する正解率を計算して表示
        print(f"test accuracy: {count/len(dataloader_test.dataset)}")

## ImageNetで事前学習したモデルを用いたファインチューニング

次に，ImageNetで事前学習したモデルを用いてCIFAR-10でファインチューニングします．

### データの準備

フルスクラッチで学習したときの画像サイズは32でしたが，ファインチューニングでは学習済みモデルに合わせるためにに224にします．画像サイズを変更する場合，パッチ数が変わる影響でPosition Embeddingのサイズを整える必要があるため，今回は省略します．

In [ ]:
img_size = 224

# 学習用データのデータ増幅と評価用データのデータ増幅の設定
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.Resize(img_size),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                     ])

test_transform  = transforms.Compose([transforms.Resize(img_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                     ])

# 学習用データと評価用データの用意
dataset_train = torchvision.datasets.CIFAR10("./", train=True, transform=train_transform, download=True)
dataset_test  = torchvision.datasets.CIFAR10("./", train=False, transform=test_transform, download=False)

# 学習用データのDataloaderと評価用データのDataloaderの用意
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=64, num_workers=2, pin_memory=True, drop_last=True)
dataloader_test  = torch.utils.data.DataLoader(dataset_test, batch_size=64, num_workers=2, pin_memory=True, drop_last=False)

### 学習条件の設定

ファインチューニングでも，ViTのSmallモデルとパラメータ数が同等のResNet-50を用います．ViTの学習済みモデルにはData-efficient image Transformers (DeiT) を使用します．ネットワーク構造は通常のViTと同様です．DeiTは学習時にRand AugmentやCutMix，MixupなどのData Augmentationを使用することで，ImageNetのみでCNNに匹敵する性能を達成した手法です．ResNet-50の学習済みモデルは，DeiTの学習条件に寄せて学習した重みです．

In [ ]:
# クラス数の設定
num_classes = 10

# ViTの定義
vit_finetune = create_model("deit_small_patch16_224", pretrained=True, num_classes=num_classes) # ImageNet (DeiT)
#vit_finetune = create_model("vit_small_patch16_224_in21k", pretrained=True, num_classes=num_classes) # ImageNet-21k

# 学習率の設定
lr  = 0.0001
# Weight Decayの設定
weight_decay = 0.05
# エポック数の設定
epochs = 5
# Warmup Epochの設定
warmup_t = 0

# 最適化手法の設定 (ViT)
optimizer_vit     = torch.optim.AdamW(vit_finetune.parameters(), lr=lr, weight_decay=weight_decay)
# 学習率のスケジューラーの設定 (ViT)
lr_scheduler_vit  = CosineLRScheduler(optimizer=optimizer_vit, t_initial=epochs, warmup_t=warmup_t)

### ViTの学習と評価

#### 参考精度

デフォルトの設定で学習を行なった場合に達成できる分類精度（目安）を記載します．

※ 学習が演習時間内に終了しない場合を考慮して記載しています．

**CNNの参考分類精度（ファインチューニング）**

```
epoch: 5, mean loss: 0.049, mean accuracy: 0.98, elapsed_time : 381.52
test accuracy: 0.9679
```

**ViTの参考分類精度（ファインチューニング）**

```
epoch: 5, mean loss: 0.016, mean accuracy: 0.99, elapsed_time : 1170.5
test accuracy: 0.9758

```

In [ ]:
# 誤差関数の設定
criterion = torch.nn.CrossEntropyLoss()

# ネットワークをGPUへ
device = torch.device("cuda")
vit_finetune.to(device)

# 学習時間短縮のためにAutomatic Mixed Precision(amp)を利用
use_amp = True
scaler_vit = torch.cuda.amp.GradScaler(enabled=use_amp)

start = time()
for epoch in range(epochs):
    # ネットワークを学習モードへ変更
    vit_finetune.train()

    # ログ用の設定
    sum_loss = 0.0
    count    = 0

    for img, cls in dataloader_train:
        # 学習用データをGPUへ
        img = img.to(device, non_blocking=True)
        cls = cls.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            # ネットワークに学習用データを入力
            logit = vit_finetune(img)
            # 損失を計算
            loss  = criterion(logit, cls)

        # ネットワークの更新
        optimizer_vit.zero_grad()
        scaler_vit.scale(loss).backward()
        scaler_vit.step(optimizer_vit)
        scaler_vit.update()

        # ログ用に損失値と正解したデータ数を取得
        sum_loss += loss.item()
        count    += torch.sum(logit.argmax(dim=1) == cls).item()

    # 学習率の調整
    lr_scheduler_vit.step(epoch)

    # ログの表示
    print(f"epoch: {epoch+1}, ", end='')
    print(f"mean loss: {round(sum_loss/len(dataloader_train), 3)}, ", end='')
    print(f"mean accuracy: {round(count/len(dataloader_train.dataset), 2)}, ", end='')
    print(f"elapsed_time : {round(time()-start, 2)}")

    # ネットワークを評価モードへ変更
    vit_finetune.eval()

    # ログ用の設定
    count = 0

    # ネットワークの評価
    with torch.no_grad():
        for img, cls in dataloader_test:
            # 評価用データをGPUへ
            img = img.to(device, non_blocking=True)
            cls = cls.to(device, non_blocking=True)

            # ネットワークに評価用データを入力
            logit = vit_finetune(img)

            # 正解したデータ数をカウント
            count += torch.sum(logit.argmax(dim=1) == cls).item()

        # 評価用データに対する正解率を計算して表示
        print(f"test accuracy: {count/len(dataloader_test.dataset)}")

# 課題
1. Multi-Head Attentionのhead数を変えてみましょう．num_headsで変更できます．ただし，埋め込み次元数（Smallモデルの場合は384）が割り切れる値にしてください．

# 参考文献

[1]  Alexey Dosovitskiy, Lucas Beyer, Alexander Kolesnikov, Dirk Weissenborn, Xiaohua Zhai, Thomas Unterthiner, Mostafa Dehghani, Matthias Minderer, Georg Heigold, Sylvain Gelly, Jakob Uszkoreit, and Neil Houlsby. An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale. In *International Conference on Learning Representations*, 2021.

# 備考（CNNの学習プログラム）

演習時間中には行いませんが，比較としてCNN(ResNet50)を用いた際の学習プログラムに記します．


## CNNの学習（スクラッチ）

In [ ]:
# クラス数の設定
num_classes = 10

# CNNの定義
cnn = create_model("resnet50", pretrained=False, num_classes=num_classes)

# 学習率の設定
lr  = 0.0005
# Weight Decayの設定
weight_decay = 0.05
# エポック数の設定
epochs = 10
# Warmup Epochの設定
warmup_t = 3

# 最適化手法の設定 (CNN)
optimizer_cnn     = torch.optim.AdamW(cnn.parameters(), lr=lr, weight_decay=weight_decay)
# 学習率のスケジューラーの設定 (CNN)
lr_scheduler_cnn  = CosineLRScheduler(optimizer=optimizer_cnn, t_initial=epochs, warmup_t=warmup_t)


# 誤差関数の設定
criterion = torch.nn.CrossEntropyLoss()

# ネットワークをGPUへ
device = torch.device("cuda")
cnn.to(device)

# 学習時間短縮のためにAutomatic Mixed Precision(amp)を利用
use_amp = True
scaler_cnn = torch.cuda.amp.GradScaler(enabled=use_amp)

start = time()
for epoch in range(epochs):
    # ネットワークを学習モードへ変更
    cnn.train()

    # ログ用の設定
    sum_loss = 0.0
    count    = 0

    for img, cls in dataloader_train:
        # 学習用データをGPUへ
        img = img.to(device, non_blocking=True)
        cls = cls.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            # ネットワークに学習用データを入力
            logit = cnn(img)
            # 損失を計算
            loss  = criterion(logit, cls)

        # ネットワークの更新
        optimizer_cnn.zero_grad()
        scaler_cnn.scale(loss).backward()
        scaler_cnn.step(optimizer_cnn)
        scaler_cnn.update()

        # ログ用に損失値と正解したデータ数を取得
        sum_loss += loss.item()
        count    += torch.sum(logit.argmax(dim=1) == cls).item()

    # 学習率の調整
    lr_scheduler_cnn.step(epoch)

    # ログの表示
    print(f"epoch: {epoch+1}, ", end='')
    print(f"mean loss: {round(sum_loss/len(dataloader_train), 3)}, ", end='')
    print(f"mean accuracy: {round(count/len(dataloader_train.dataset), 2)}, ", end='')
    print(f"elapsed_time : {round(time()-start, 2)}")

    # ネットワークを評価モードへ変更
    cnn.eval()

    # ログ用の設定
    count = 0

    # ネットワークの評価
    with torch.no_grad():
        for img, cls in dataloader_test:
            # 評価用データをGPUへ
            img = img.to(device, non_blocking=True)
            cls = cls.to(device, non_blocking=True)

            # ネットワークに評価用データを入力
            logit = cnn(img)

            # 正解したデータ数をカウント
            count += torch.sum(logit.argmax(dim=1) == cls).item()

        # 評価用データに対する正解率を計算して表示
        print(f"    test accuracy: {count/len(dataloader_test.dataset)}")

## CNNの学習（ファインチューニング）

In [ ]:
# クラス数の設定
num_classes = 10

# CNNの定義
cnn_finetune = create_model("resnet50", pretrained=True, num_classes=num_classes)

# 学習率の設定
lr  = 0.0001
# Weight Decayの設定
weight_decay = 0.05
# エポック数の設定
epochs = 5
# Warmup Epochの設定
warmup_t = 0

# 最適化手法の設定 (CNN)
optimizer_cnn     = torch.optim.AdamW(cnn_finetune.parameters(), lr=lr, weight_decay=weight_decay)
# 学習率のスケジューラーの設定 (CNN)
lr_scheduler_cnn  = CosineLRScheduler(optimizer=optimizer_cnn, t_initial=epochs, warmup_t=warmup_t)


# 誤差関数の設定
criterion = torch.nn.CrossEntropyLoss()

# ネットワークをGPUへ
device = torch.device("cuda")
cnn_finetune.to(device)

# 学習時間短縮のためにAutomatic Mixed Precision(amp)を利用
use_amp = True
scaler_cnn = torch.cuda.amp.GradScaler(enabled=use_amp)

start = time()
for epoch in range(epochs):
    # ネットワークを学習モードへ変更
    cnn_finetune.train()

    # ログ用の設定
    sum_loss = 0.0
    count    = 0

    for img, cls in dataloader_train:
        # 学習用データをGPUへ
        img = img.to(device, non_blocking=True)
        cls = cls.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            # ネットワークに学習用データを入力
            logit = cnn_finetune(img)
            # 損失を計算
            loss  = criterion(logit, cls)

        # ネットワークの更新
        optimizer_cnn.zero_grad()
        scaler_cnn.scale(loss).backward()
        scaler_cnn.step(optimizer_cnn)
        scaler_cnn.update()

        # ログ用に損失値と正解したデータ数を取得
        sum_loss += loss.item()
        count    += torch.sum(logit.argmax(dim=1) == cls).item()

    # 学習率の調整
    lr_scheduler_cnn.step(epoch)

    # ログの表示
    print(f"epoch: {epoch+1}, ", end='')
    print(f"mean loss: {round(sum_loss/len(dataloader_train), 3)}, ", end='')
    print(f"mean accuracy: {round(count/len(dataloader_train.dataset), 2)}, ", end='')
    print(f"elapsed_time : {round(time()-start, 2)}")

    # ネットワークを評価モードへ変更
    cnn_finetune.eval()

    # ログ用の設定
    count = 0

    # ネットワークの評価
    with torch.no_grad():
        for img, cls in dataloader_test:
            # 評価用データをGPUへ
            img = img.to(device, non_blocking=True)
            cls = cls.to(device, non_blocking=True)

            # ネットワークに評価用データを入力
            logit = cnn_finetune(img)

            # 正解したデータ数をカウント
            count += torch.sum(logit.argmax(dim=1) == cls).item()

        # 評価用データに対する正解率を計算して表示
        print(f"test accuracy: {count/len(dataloader_test.dataset)}")